In [ ]:
import os
import PyPDF2
import extract_msg
from shutil import copy
from extract_msg.exceptions import InvalidFileFormatError

#set time to us central
os.environ['TZ'] = 'US/Central'

def get_last_3_parts(file_path):
    return "/".join(file_path.split(os.path.sep)[-4:])

def getAttachments(reader):
    #initialize empty dict to store extracted document names and data:
    attachments = {}
    #Pdf catalog fetch, high level metadata
    catalog = reader.trailer["/Root"]
    if "/Names" in catalog:
        if "/EmbeddedFiles" in catalog["/Names"]:
            fileNames = catalog['/Names']['/EmbeddedFiles']['/Names']
            for f in fileNames:
                if isinstance(f, str):
                    name = f
                    dataIndex = fileNames.index(f) + 1
                    fDict = fileNames[dataIndex].get_object()
                    if "/EF" in fDict:
                        fData = fDict["/EF"]["/F"].get_data()
                        attachments[name] = fData
                    elif "/F" in fDict:
                        embedded_file = PyPDF2.PdfReader(fDict['F'].getData())
                        embedded_attachments = getAttachments(embedded_file)
                        attachments.update(embedded_attachments)

        for pagenum in range(0, len(reader.pages)):
            page_object = reader.pages[pagenum]
            if "/Annots" in page_object:
                for annot in page_object['/Annots']:
                    annotobj = annot.get_object()
                    if annotobj['/Subtype'] == '/FileAttachment':
                        fileobj = annotobj["/FS"]
                        if "/EF" in fileobj:
                            attachments[fileobj["/F"]] = fileobj["/EF"]["/F"].get_data()
                        elif "/F" in fileobj:
                            embedded_file = PyPDF2.PdfReader(fileobj["/"]).get_data()
                            embedded_attachments = getAttachments(embedded_file)
                            attachments.update(embedded_attachments)
    return attachments

def extract_attachments(src_folder, dest_folder):
    if not os.path.exists(dest_folder):
        os.makedirs(dest_folder)

    # iterate through all files in the source folder:
    for root, dirs, files in os.walk(src_folder):
        for file_name in files:
            # create the file path and the new folder path for the current file
            file_path = os.path.join(root, file_name)
            new_folder_name = os.path.splitext(file_name)[0]
            new_folder = os.path.join(dest_folder, new_folder_name)

            # check if the current file is a PDF file. if so, open it, read the attachments and copy attach to dest folder
            try:
                if file_name.lower().endswith('.pdf'):
                    with open(file_path, 'rb') as pdf_file:
                        pdf_reader = PyPDF2.PdfReader(pdf_file)
                        attachments = getAttachments(pdf_reader)

                        # Copy the PDF file to the new folder
                        for attachment_name, attachment_data in attachments.items():
                            if not os.path.exists(new_folder):
                                os.makedirs(new_folder)

                            copy(file_path, new_folder)

                            with open(os.path.join(new_folder, attachment_name), 'wb') as f:
                                f.write(attachment_data)

                elif file_name.lower().endswith('.msg'):
                    try:
                        msg = extract_msg.Message(file_path)
                        msg_attachments = msg.attachments

                        # Copy the MSG file to the new folder
                        copy(file_path, new_folder)

                        for attachment in msg_attachments:
                            if not os.path.exists(new_folder):
                                os.makedirs(new_folder)

                            copy(file_path, new_folder)

                            attachment.save(customPath=os.path.join(new_folder, attachment.longFilename))

                    except NotImplementedError:
                        print(f"Unsupported Container type in: \n{get_last_3_parts(file_path)}")
                    except InvalidFileFormatError:
                        print(f"Invalid or Corrupted File in: \n{get_last_3_parts(file_path)}")

                else:
                    print(f"Skipping unsupported file: {file_name}")

                    # copy other file types to the new folder
                    copy(file_path, new_folder)

            except FileNotFoundError:
                print(f"Warning: File Not found or inaccessible: \n{get_last_3_parts(file_path)}")
src_folder =
dest_folder =
extract_attachments(src_folder, dest_folder)

SyntaxError: invalid syntax (2830201818.py, line 1)

In [2]:
import os
print(os.getcwd())


C:\Users\halis
